In [145]:
import pandas as pd
Source1=pd.read_excel('Sales.xlsx',sheet_name = 'Distributor 1')
Source2=pd.read_excel('Sales.xlsx',sheet_name = 'Distributor 2' )

In [146]:
Source1['Distributor'] = 'Distributor 1'
Source2['Distributor'] = 'Distributor 2'

In [147]:
Data=pd.concat([Source1,Source2])
Data.head()

,Date,Item Code,Item Name,Product Line,Client Code,Client Name,Region,QTY,Sales,Distributor
0,2019-10-07,520235,Product 1,Product Line 1,426599,Customer 1,Cairo,2.0,14.4,Distributor 1
1,2019-10-07,520235,Product 1,Product Line 1,432152,Customer 2,Cairo,2.0,14.4,Distributor 1
2,2019-10-07,520235,Product 1,Product Line 1,334071,Customer 3,Cairo,2.0,14.4,Distributor 1
3,2019-10-07,520235,Product 1,Product Line 1,405758,Customer 4,Cairo,50.0,360.0,Distributor 1
4,2019-10-07,520235,Product 1,Product Line 1,405670,Customer 5,Cairo,5.0,36.0,Distributor 1


In [148]:
Data['Date']=pd.to_datetime(Data['Date'])
Data['Day']=Data['Date'].dt.day
Data['Month']=Data['Date'].dt.month
Data['Year']=Data['Date'].dt.year
Data['Month']=Data['Month'].replace([1,2,3,4,5,6,7,8,9,10,11,12],['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
Data['Quarter']=Data['Date'].dt.quarter
Data['Quarter']=Data['Quarter'].replace([1,2,3,4],['Q1','Q2','Q3','Q4'])

In [149]:
Data.head()

,Date,Item Code,Item Name,Product Line,Client Code,Client Name,Region,QTY,Sales,Distributor,Day,Month,Year,Quarter
0,2019-10-07,520235,Product 1,Product Line 1,426599,Customer 1,Cairo,2.0,14.4,Distributor 1,7,Oct,2019,Q4
1,2019-10-07,520235,Product 1,Product Line 1,432152,Customer 2,Cairo,2.0,14.4,Distributor 1,7,Oct,2019,Q4
2,2019-10-07,520235,Product 1,Product Line 1,334071,Customer 3,Cairo,2.0,14.4,Distributor 1,7,Oct,2019,Q4
3,2019-10-07,520235,Product 1,Product Line 1,405758,Customer 4,Cairo,50.0,360.0,Distributor 1,7,Oct,2019,Q4
4,2019-10-07,520235,Product 1,Product Line 1,405670,Customer 5,Cairo,5.0,36.0,Distributor 1,7,Oct,2019,Q4


In [115]:
from sqlalchemy import create_engine
connection_string = "mssql+pyodbc://localhost/Sales_DWH?driver=ODBC+Driver+17+for+SQL+Server?Trusted_Connection=yes"
engine = create_engine(connection_string)

In [151]:
Data.rename(columns = {"Client Name": "Client_Name"}, inplace = True)
Data.rename(columns = {"Client Code": "Client_Code"}, inplace = True)
Data.rename(columns = {"Product Line": "Product_Line"}, inplace = True)
Data.rename(columns = {"Distributor": "Distributor_Name"}, inplace = True)
Data.rename(columns = {"Item Code": "Item_Code"}, inplace = True)
Data.rename(columns = {"Item Name": "Item_Name"}, inplace = True)
Data.rename(columns = {"Date": "Full_Date"}, inplace = True)
Data.rename(columns = {"Day": "Day_Of_Month"}, inplace = True)
Data.rename(columns = {"QTY": "Quantity"}, inplace = True)
Data.head()

,Full_Date,Item_Code,Item_Name,Product_Line,Client_Code,Client_Name,Region,Quantity,Sales,Distributor_Name,Day_Of_Month,Month,Year,Quarter
0,2019-10-07,520235,Product 1,Product Line 1,426599,Customer 1,Cairo,2.0,14.4,Distributor 1,7,Oct,2019,Q4
1,2019-10-07,520235,Product 1,Product Line 1,432152,Customer 2,Cairo,2.0,14.4,Distributor 1,7,Oct,2019,Q4
2,2019-10-07,520235,Product 1,Product Line 1,334071,Customer 3,Cairo,2.0,14.4,Distributor 1,7,Oct,2019,Q4
3,2019-10-07,520235,Product 1,Product Line 1,405758,Customer 4,Cairo,50.0,360.0,Distributor 1,7,Oct,2019,Q4
4,2019-10-07,520235,Product 1,Product Line 1,405670,Customer 5,Cairo,5.0,36.0,Distributor 1,7,Oct,2019,Q4


In [108]:
for _, row in Data.iterrows():
    with engine.begin() as connection:

        #Customer_key
        query_check = '''
            SELECT Customer_key
            FROM Customer_Dimension
            WHERE Client_Name = ? AND Client_Code = ?;'''
        existing_customer = connection.execute(query_check, row['Client_Name'], row['Client_Code']).fetchone()
        if not existing_customer:
            query_insert = '''
                INSERT INTO Customer_Dimension (Client_Name, Client_Code)
                OUTPUT INSERTED.Customer_key
                VALUES (?, ?);'''
            result = connection.execute(query_insert, row['Client_Name'], row['Client_Code'])
            Customer_key = int(result.scalar())
        else:
            Customer_key = existing_customer[0]
        Data.loc[(Data['Client_Name'] == row['Client_Name']) & (Data['Client_Code'] == row['Client_Code']), 'Customer_key'] = Customer_key


        #Region_key
        query_check = '''
            SELECT Region_Key
            FROM Region_Dimension
            WHERE Region = ? ;'''
        existing_Region = connection.execute(query_check, row['Region']).fetchone()
        if not existing_Region:
            query_insert = '''
                INSERT INTO Region_Dimension (Region)
                OUTPUT INSERTED.Region_Key
                VALUES (?);'''
            result = connection.execute(query_insert, row['Region'])
            Region_Key = int(result.scalar())
        else:
            Region_Key = existing_Region[0]
        Data.loc[(Data['Region'] == row['Region']), 'Region_Key'] = Region_Key


        #Product_Line_Key
        query_check = '''
            SELECT Product_Line_Key
            FROM Product_Line_Dimension
            WHERE Product_Line = ? ;'''
        existing_Product_Line = connection.execute(query_check, row['Product_Line']).fetchone()
        if not existing_Product_Line:
            query_insert = '''
                INSERT INTO Product_Line_Dimension (Product_Line)
                OUTPUT INSERTED.Product_Line_Key
                VALUES (?);'''
            result = connection.execute(query_insert, row['Product_Line'])
            Product_Line_Key = int(result.scalar())
        else:
            Product_Line_Key = existing_Product_Line[0]
        Data.loc[(Data['Product_Line'] == row['Product_Line']), 'Product_Line_Key'] = Product_Line_Key


        #Distributor_Key
        query_check = '''
            SELECT Distributor_Key
            FROM Distributor_Dimension
            WHERE Distributor_Name = ? ;'''
        existing_Distributor_Name = connection.execute(query_check, row['Distributor_Name']).fetchone()
        if not existing_Distributor_Name:
            query_insert = '''
                INSERT INTO Distributor_Dimension (Distributor_Name)
                OUTPUT INSERTED.Distributor_Key
                VALUES (?);'''
            result = connection.execute(query_insert, row['Distributor_Name'])
            Distributor_Key = int(result.scalar())
        else:
            Distributor_Key = existing_Distributor_Name[0]
        Data.loc[(Data['Distributor_Name'] == row['Distributor_Name']), 'Distributor_Key'] = Distributor_Key


        #Product_Key
        query_check = '''
            SELECT Product_Key
            FROM Product_Dimension
            WHERE Item_Name = ? AND Item_Code = ?;'''
        existing_Product_Key = connection.execute(query_check, row['Item_Name'], row['Item_Code']).fetchone()
        if not existing_Product_Key:
            query_insert = '''
                INSERT INTO Product_Dimension (Item_Name, Item_Code)
                OUTPUT INSERTED.Product_Key
                VALUES (?, ?);'''
            result = connection.execute(query_insert, row['Item_Name'], row['Item_Code'])
            Product_Key = int(result.scalar())
        else:
            Product_Key = existing_Product_Key[0]
        Data.loc[(Data['Item_Name'] == row['Item_Name']) & (Data['Item_Code'] == row['Item_Code']), 'Product_Key'] = Product_Key


        #Date_Key
        query_check = '''
            SELECT Date_Key
            FROM Date_Dimension
            WHERE Full_Date = ? AND Day_Of_Month = ? AND Month = ? AND Quarter = ? AND Year = ?;'''
        existing_Date_Key = connection.execute(query_check, row['Full_Date'], row['Day_Of_Month'], row['Month'], row['Quarter'], row['Year']).fetchone()
        if not existing_Date_Key:
            query_insert = '''
                INSERT INTO Date_Dimension (Full_Date, Day_Of_Month, Month, Quarter, Year)
                OUTPUT INSERTED.Date_Key
                VALUES (?, ?, ?, ?, ?);'''
            result = connection.execute(query_insert, row['Full_Date'], row['Day_Of_Month'], row['Month'], row['Quarter'], row['Year'])
            Date_Key = int(result.scalar())
        else:
            Date_Key = existing_Date_Key[0]
        Data.loc[(Data['Full_Date'] == row['Full_Date']) & (Data['Day_Of_Month'] == row['Day_Of_Month']) & (Data['Month'] == row['Month']) & (Data['Quarter'] == row['Quarter']) & (Data['Year'] == row['Year']), 'Date_Key'] = Date_Key

engine.dispose()

In [139]:
Data['Region_Key']=Data['Region_Key'].astype(int)
Data['Product_Line_Key']=Data['Product_Line_Key'].astype(int)
Data['Distributor_Key']=Data['Distributor_Key'].astype(int)
Data['Product_Key']=Data['Product_Key'].astype(int)
Data['Date_Key']=Data['Date_Key'].astype(int)
Data.head() 

,Full_Date,Item_Code,Item_Name,Product_Line,Client_Code,Client_Name,Region,Quantity,Sales,Distributor_Name,Day_Of_Month,Month,Year,Quarter,Customer_key,Region_Key,Product_Line_Key,Distributor_Key,Product_Key,Date_Key
0,2019-10-07,520235,Product 1,Product Line 1,426599,Customer 1,Cairo,2.0,14.4,Distributor 1,7,Oct,2019,Q4,163391,8,5,3,89,2
1,2019-10-07,520235,Product 1,Product Line 1,432152,Customer 2,Cairo,2.0,14.4,Distributor 1,7,Oct,2019,Q4,163401,8,5,3,89,2
2,2019-10-07,520235,Product 1,Product Line 1,334071,Customer 3,Cairo,2.0,14.4,Distributor 1,7,Oct,2019,Q4,163411,8,5,3,89,2
3,2019-10-07,520235,Product 1,Product Line 1,405758,Customer 4,Cairo,50.0,360.0,Distributor 1,7,Oct,2019,Q4,163421,8,5,3,89,2
4,2019-10-07,520235,Product 1,Product Line 1,405670,Customer 5,Cairo,5.0,36.0,Distributor 1,7,Oct,2019,Q4,163431,8,5,3,89,2


In [143]:
Fact_Table = Data[['Date_Key','Product_Key','Customer_key','Product_Line_Key','Region_Key','Distributor_Key','Quantity','Sales']]
Fact_Table.to_sql('Fact_Table', engine, if_exists='append', index=False)
Fact_Table.head()

,Date_Key,Product_Key,Customer_key,Product_Line_Key,Region_Key,Distributor_Key,Quantity,Sales
0,2,89,163391,5,8,3,2.0,14.4
1,2,89,163401,5,8,3,2.0,14.4
2,2,89,163411,5,8,3,2.0,14.4
3,2,89,163421,5,8,3,50.0,360.0
4,2,89,163431,5,8,3,5.0,36.0
